In [1]:
import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py

from normalizer import *
from self_similar import *
prepare_gap_env()

In [3]:

def permute(space, repeat) -> str:
    """ Get all the words of letters from the given
    space of given length.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    repeat  : int, length of yielded words
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False
    Yields
    -------
    str, result word
    """
    if repeat == 1:
        for el in space:
            yield [el]
    elif repeat < 1:
        yield []
    else:
        for prev in permute(space, repeat - 1):
            for el in space:
                yield prev + [el]


def all_words(space, max_len=-1):
    """ Get all possible words (infinite set) of elements from the given
    space.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False.
    max_len : maximum allowed length of returned words.
              Default None, which means that there is no maximum length

    Yields
    -------
    str, result word
    """
    i = 1
    while True:
        for el in permute(space, repeat=i):
            yield el
        i += 1

        if max_len != -1 and i > max_len:
            break

def build_group(group_index, deep=3):
    e = matrix(QQ, [[1, 0], [0, 1]])
    G = to_L_basis(group_index, dim=2)
    gens = G.GeneratorsOfGroup()
    gens = [matrix(QQ, el) for el in gens]
    gens = [el for el in gens if el[:2, :2] != e]

    e = matrix(QQ, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    alpha = {}
    snot = []
    for seq in all_words(gens, max_len=deep): 
        
        el = e 
        for el2 in seq: 
            el *= el2 
        
        g = el[:2, :2]
        t = el[:2, 2]
        if str(g) not in alpha:
            alpha[str(g)] = t
            snot.append(block_matrix([[g, t], [0, 1]]))

    P = G.PointGroup()
    P = [matrix(QQ, el) for el in P.AsList()]

    for mtx in P:
        assert str(mtx) in alpha, 'couldnt build every element of point group'

    return G, P, alpha, snot

In [5]:
G, P, alpha, snot = build_group(7)
latex(snot)

\left[\left(\begin{array}{rr|r}
-1 & 0 & 0 \\
0 & -1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right), \left(\begin{array}{rr|r}
-1 & 0 & \frac{1}{2} \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right), \left(\begin{array}{rr|r}
1 & 0 & 0 \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right), \left(\begin{array}{rr|r}
1 & 0 & -\frac{1}{2} \\
0 & -1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)\right]